In [3]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [4]:
def local_regression(x0, X, Y, tau):
    # add bias term
    x0 = np.r_[1.0, x0]    
    X = np.c_[np.ones(len(X)), X]
    
    # fit model: normal equations with kernel      
    w = radial_kernel(x0, X, tau) 
   
    m = len(X)
    xw = X.T * w
    beta = np.linalg.pinv(xw @ X) @ xw @ Y
    
    gradient = Gradient(beta, X, Y, xw, m)
    # predict value
    return x0 @ beta, beta, gradient


def Cost(theta,x,Y,W,m):
    cost = ((x @ theta-Y)**2)
    return cost/m

def Gradient(theta,x,Y,W,m):
    gradient = (((x @ theta-Y) @ x)/m)
    return gradient

def Accuracy(theta,X, Y):
    #score_predicted = 1-((y-y_gd)**2).sum()/((y-y.mean())**2).sum()    
    return ((Y - X @ theta)**2).sum()/((Y - Y.mean())**2).sum()

def cost_w(w,y,h_theta):
    return np.sum(w*(y-h_theta))

def radial_kernel(x0, X, tau):
    w =  np.exp(np.sum((X - x0) ** 2, axis=1) / (-2 * tau * tau))
    #w_1 = np.exp(-np.dot((x0 - X).T, (x0-X))/2*tau**2)    
    return w

def polynomial(x,degree):
    #poly = np.zeros((1,2))
    poly = []
    for i in np.arange(1,degree+1,1):
        poly.append(x**i)
        #poly = np.concatenate((poly,x**(i)), axis = 1)
    return poly

In [5]:
m = 300


# generate dataset
#X = np.sort(np.random.uniform(0, 6,m))
X = np.linspace(0,6,m)
Y = np.sin(X*np.pi/2) + np.random.normal(0,.25,m)


#X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)
idx = np.arange(0,len(X), 1)
np.random.shuffle(idx)
idx_train = np.sort(idx[:int(len(idx)*.8)])
idx_test =np.sort(idx[:int(len(idx)*.2)])

X_train = X[idx_train]
X_test = X[idx_test]
Y_train = Y[idx_train]
Y_test = Y[idx_test]

X_train_fit  = np.array(polynomial(X_train, 1)).T
X_test_fit  = np.array(polynomial(X_test, 1)).T

In [6]:
def compute_lwr(X, Y, tau):
    prediction_arr = []
   
    gradient_arr = []
    #accuracy = []
    theta_arr = []
    #choose data points at regular intervals
    
    ix = np.arange(0, len(X_test),1)    
    for i in ix:
       prediction, beta, gradient = local_regression(X_test[i],X, Y, tau)
       prediction_arr.append(prediction)       
       gradient_arr.append(gradient)
       #accuracy.append(local_regression(X[i],X, Y, tau)[2])
       theta_arr.append(beta)
    return prediction_arr,theta_arr, gradient_arr      

In [8]:
prediction1, theta1, gradient1 = compute_lwr(X_train_fit,Y_train,10)
prediction2, theta2, gradient2 = compute_lwr(X_train_fit,Y_train,1)
prediction3, theta3, gradient3 = compute_lwr(X_train_fit,Y_train,.2)
prediction4, theta4, gradient4 = compute_lwr(X_train_fit,Y_train,.01)   

tau = .05
prediction_arr_train = []
prediction_arr_test = []
theta_arr_train = []
tau_arr_train = []
gradient_arr_train = []

while tau < 2.0:
    prediction, theta, gradient = compute_lwr(X_train_fit,Y_train,tau)
    gradient_arr_train.append(gradient)
    theta_arr_train.append(theta)
    prediction_arr_train.append(np.sum(prediction))   
    tau_arr_train.append(tau)    
    tau = tau + tau*.01

In [9]:
# fig1, ax = plt.subplots(2,2)
# fig1.set_figheight(5)
# fig1.set_figwidth(10)
# fig1.subplots_adjust(left=.2, bottom=None, right=None, top=None, wspace=.4, hspace=.4)
# plt1 = plt.subplot(2,2,1)
# plt2 = plt.subplot(2,2,2)
# plt3 = plt.subplot(2,2,3)
# plt4 = plt.subplot(2,2,4)

# plt1.plot(X,Y,'c.')

# plt1.plot(X_test,prediction1,'g.')
# plt1.set_title('Prediction 1, Tau = 10')
# plt1.set_xlabel('units')
# plt1.set_ylabel('Frequency')
# plt1.grid(axis='both', alpha=.25)

# plt2.plot(X,Y,'c.')
# plt2.set_title('Prediction 2, Tau = 1')
# plt2.plot(X_test,prediction2,'g.')
# plt2.set_xlabel('units')
# plt2.set_ylabel('Frequency')
# plt2.grid(axis='both', alpha=.25)

# plt3.plot(X,Y,'c.')
# plt3.set_title('Prediction 3, Tau = .2')
# plt3.plot(X_test,prediction3,'g.')
# plt3.set_xlabel('units')
# plt3.set_ylabel('Frequency')
# plt3.grid(axis='both', alpha=.25)


# plt4.plot(X,Y,'c.')
# plt4.set_title('Prediction 4, Tau = .01')
# plt4.plot(X_test,prediction4,'g.')
# plt4.set_xlabel('units')
# plt4.set_ylabel('Frequency')
# plt4.grid(axis='both', alpha=.25)

# plt.show()

In [14]:
# fig3 = plt.figure()
# ax = plt.axes()
# plt.title('Tau vs prediction error')
# plt.xlabel('Tau')
# plt.grid(axis='both', alpha=.25)
# plt.plot(tau_arr_train,prediction_arr_train,'g-')
# plt.show()

tau_optimal = tau_arr_train[np.argmin(prediction_arr_train)]
prediction5, theta5, gradient5 = compute_lwr(X_train_fit,Y_train,tau_optimal)
# prediction_arr_train
# a = np.array([0,1,2,7,-4])
# np.argmin(a)
# fig4 = plt.figure()
# ax = plt.axes()
# plt.title('Locally weighted Linear Regression')
# plt.grid(axis='both', alpha=.25)
# plt.plot(X_train,Y_train,'c.')
# plt.plot(X_test,prediction5,'b.')
# plt.show()
# print('Optimal value of Tau ', tau_optimal)

4